In [1]:
import requests
from lxml import etree
import pandas as pd
import re
import numpy as np
import math

#首先定义全局变量，这些全局变量主要包括股票代码和所需年份

In [2]:
stockid='600519'
year=2018
Tax=0.25

In [3]:
#定义一个将千分位分隔符去掉的函数，它需要在爬虫程序中用到
def convert(num):
    if num=='':
        return None
    elif num==0:
        return 0
    else:
        return float(re.sub(',','',num))

#由于要多次使用此爬虫程序，因此将爬虫程序定义成一个函数对象,函数如下

def sheet1(url,xpath1):#参数分别为网址，xpath信息提取路径
    header={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36 SE 2.X MetaSr 1.0'}
    response=requests.get(url,headers=header)#利用requests库，使用get方式获取网页文档
    response.encoding='gb2312'#指定解析网页的编码方式，避免网页中文乱码
    html=etree.HTML(response.text)
    FD_BalanceSheet=html.xpath(xpath1)#获取当前网页下的整个报表数据
    Accounts=[]
    values=[]
    for i in range(2,len(FD_BalanceSheet)):
        sheet1=FD_BalanceSheet[i].xpath('./td//text()')
        if len(sheet1)==1:
            sheet1.insert(1,'')#有部分属于总科目，比如流动资产，非流动资产，为了避免报错，将这类总科目后的数值设为空白
        if sheet1[1]=='--':#将原报表中不存在的科目转化为0
            sheet1[1]=0
        values1=convert(sheet1[1])
        Accounts.append(sheet1[0])
        values.append(values1)
#    vFD_BalanceSheet=pd.DataFrame({'Accounts':Accounts,'values':values})#将结果以Dataframe的方式输出,其中空白部分的数据会被转化成NaN缺失值形式
    return Accounts,values

In [4]:
yearvalues=[]
for i in range(year-2,year+1):
    capitalurl='http://money.finance.sina.com.cn/corp/go.php/vFD_BalanceSheet/stockid/'+str(stockid)+'/ctrl/'+str(i)+'/displaytype/4.phtml'
    capitalxpath1='//*[@id="BalanceSheetNewTable0"]/tbody//tr'
    Accounts,years=sheet1(capitalurl,capitalxpath1)#获取近三年的资产负债表表
    yearvalues.append(years)
data1=pd.DataFrame({'Accounts':Accounts})
data1[year-2]=yearvalues[0]
data1[year-1]=yearvalues[1]
data1[year]=yearvalues[2]
data1

,Accounts,2016,2017,2018
0,流动资产,NaN,NaN,NaN
1,货币资金,6685496.21,8786886.99,11207479.14
2,交易性金融资产,0.00,0.00,0.00
3,衍生金融资产,0.00,0.00,0.00
4,应收票据,81762.72,122170.60,56373.97
5,应收账款,0.00,0.00,0.00
6,预付款项,104610.07,79080.73,118237.85
7,应收利息,14090.49,24145.86,34388.99
8,应收股利,0.00,0.00,0.00
9,其他应收款,7722.76,3132.35,5000.05


In [5]:
#获取近三年的利润表
yearvalues1=[]
for i in range(year-2,year+1):
    profiturl='http://money.finance.sina.com.cn/corp/go.php/vFD_ProfitStatement/stockid/'+str(stockid)+'/ctrl/'+str(i)+'/displaytype/4.phtml'
    profitxpath1='//*[@id="ProfitStatementNewTable0"]/tbody//tr'
    Accounts,values=sheet1(profiturl,profitxpath1)
    yearvalues1.append(values)
vFD_ProfitStatement=pd.DataFrame({'Accounts':Accounts})
vFD_ProfitStatement[year-2]=yearvalues1[0]
vFD_ProfitStatement[year-1]=yearvalues1[1]
vFD_ProfitStatement[year]=yearvalues1[2]
vFD_ProfitStatement

,Accounts,2016,2017,2018
0,一、营业总收入,4015508.44,6106275.69,7719938.41
1,营业收入,3886219.00,5821786.13,7363887.24
2,二、营业总成本,1588945.92,2212274.93,2586603.06
3,营业成本,341010.41,594043.64,652292.18
4,营业税金及附加,650892.63,840421.45,1128892.68
5,销售费用,168105.20,298606.85,257207.69
6,管理费用,418718.98,472054.28,532594.08
7,财务费用,-3317.52,-5572.23,-352.12
8,资产减值损失,1232.75,-805.37,128.97
9,公允价值变动收益,0.00,0.00,0.00


In [6]:
#获取近三年时间的现金流量表
cashvalues=[]
for i in range(year-2,year+1):
    cashflowurl='http://money.finance.sina.com.cn/corp/go.php/vFD_CashFlow/stockid/'+str(stockid)+'/ctrl/'+str(i)+'/displaytype/4.phtml'
    cashxpath='//*[@id="ProfitStatementNewTable0"]/tbody//tr'
    Accounts,cash=sheet1(cashflowurl,cashxpath)
    cashvalues.append(cash)
vFD_CashFlow=pd.DataFrame({'Accounts':Accounts})
vFD_CashFlow[year-2]=cashvalues[0]
vFD_CashFlow[year-1]=cashvalues[1]
vFD_CashFlow[year]=cashvalues[2]
vFD_CashFlow

,Accounts,2016,2017,2018
0,一、经营活动产生的现金流量,NaN,NaN,NaN
1,销售商品、提供劳务收到的现金,6101296.41,6442147.93,8426869.57
2,收到的税费返还,0.00,0.00,0.00
3,收到的其他与经营活动有关的现金,18914.27,54216.22,62155.84
4,经营活动现金流入小计,6727914.56,6736946.25,8934563.54
5,购买商品、接受劳务支付的现金,277302.04,487576.85,529851.80
6,支付给职工以及为职工支付的现金,467415.42,548960.61,665313.77
7,支付的各项税费,1751051.63,2306564.85,3203217.81
8,支付的其他与经营活动有关的现金,237148.68,294029.64,293576.68
9,经营活动现金流出小计,2982789.60,4521642.64,4796040.10


In [7]:
#将需要的营业收入，营业成本，管理费用，销售费用取出
everyvalues=vFD_ProfitStatement.loc[(vFD_ProfitStatement['Accounts']=='营业收入')|
                                    (vFD_ProfitStatement['Accounts']=='营业成本')|
                                   (vFD_ProfitStatement['Accounts']=='销售费用')|
                                   (vFD_ProfitStatement['Accounts']=='管理费用')]
everyvalues1=np.array(everyvalues.iloc[:,[1,2,3]])
total=pd.DataFrame({'营业收入':everyvalues1[0],
                    '营业成本':everyvalues1[1],
                    '销售费用':everyvalues1[2],
                    '管理费用':everyvalues1[3]},
                   index=[year-2,year-1,year])
total

,营业收入,营业成本,销售费用,管理费用
2016,3886219.00,341010.41,168105.20,418718.98
2017,5821786.13,594043.64,298606.85,472054.28
2018,7363887.24,652292.18,257207.69,532594.08


In [8]:
#计算折旧与摊销的总额，并将其合并到上述表格中
Amort=vFD_CashFlow[(vFD_CashFlow['Accounts']=='固定资产折旧、油气资产折耗、生产性物资折旧')|
                   (vFD_CashFlow['Accounts']=='无形资产摊销')|
                   (vFD_CashFlow['Accounts']=='长期待摊费用摊销')]
#计算三者每一年的总和
Amort1=Amort.sum(axis=0)
Amort2=list(Amort1[1:])
total['折旧与摊销']=Amort2
total

,营业收入,营业成本,销售费用,管理费用,折旧与摊销
2016,3886219.00,341010.41,168105.20,418718.98,93419.47
2017,5821786.13,594043.64,298606.85,472054.28,112583.45
2018,7363887.24,652292.18,257207.69,532594.08,117542.55


In [9]:
#计算不包括折旧与摊销的营业成本
total['不包括折旧与摊销的营业成本']=total['营业成本']-total['折旧与摊销']
#计算销售与管理费用总和
total['销售管理以及行政费用']=total['销售费用']+total['管理费用']
#计算息前税前折旧摊销前利润（EBITDA）
total['息前税前折旧摊销前利润（EBITDA）']=total['营业收入']-total['不包括折旧与摊销的营业成本']-total['销售管理以及行政费用']
total['EBIT']=total['息前税前折旧摊销前利润（EBITDA）']-total['折旧与摊销']
total['息前税后利润']=total['EBIT']*(1-Tax)
total#这个表是为了查看各个指标的情况

,营业收入,营业成本,销售费用,管理费用,折旧与摊销,不包括折旧与摊销的营业成本,销售管理以及行政费用,息前税前折旧摊销前利润（EBITDA）,EBIT,息前税后利润
2016,3886219.00,341010.41,168105.20,418718.98,93419.47,247590.94,586824.18,3051803.88,2958384.41,2.218788e+06
2017,5821786.13,594043.64,298606.85,472054.28,112583.45,481460.19,770661.13,4569664.81,4457081.36,3.342811e+06
2018,7363887.24,652292.18,257207.69,532594.08,117542.55,534749.63,789801.77,6039335.84,5921793.29,4.441345e+06


In [10]:
#接下来计算预测期（预测期为10年）的息前税后利润
#计算营业收入前三年的增长率以及其均值
total1=total.loc[:,['营业收入','不包括折旧与摊销的营业成本','销售管理以及行政费用','折旧与摊销']]
total1['营业收入增长率']=(total1['营业收入']/total1['营业收入'].shift(1))-1
total1['不包括折旧与摊销的营业成本占营业收入']=total1['不包括折旧与摊销的营业成本']/total1['营业收入']
total1['销售管理费用占营业收入']=total1['销售管理以及行政费用']/total1['营业收入']
total1['折旧摊销占营业收入']=total1['折旧与摊销']/total1['营业收入']
total2=total1.loc[:,['营业收入','不包括折旧与摊销的营业成本','销售管理以及行政费用','折旧与摊销']]
total2#此表将会作为总体进行预测

,营业收入,不包括折旧与摊销的营业成本,销售管理以及行政费用,折旧与摊销
2016,3886219.00,247590.94,586824.18,93419.47
2017,5821786.13,481460.19,770661.13,112583.45
2018,7363887.24,534749.63,789801.77,117542.55


In [11]:
meanprofit=total1['营业收入增长率'].mean()
meancost=total1['不包括折旧与摊销的营业成本占营业收入'].mean()
meanfee=total1['销售管理费用占营业收入'].mean()
meanzhe=total1['折旧摊销占营业收入'].mean()

In [12]:
#资本性支出，即当期固定资产投资净额，在现金流量表中的投资活动列示
Assetsinvestment=vFD_CashFlow[vFD_CashFlow['Accounts']=='购建固定资产、无形资产和其他长期资产所支付的现金']
Assetsinvestment1=np.array(Assetsinvestment.iloc[:,[1,2,3]])
total2['资本性支出']=Assetsinvestment1[0]
total2

,营业收入,不包括折旧与摊销的营业成本,销售管理以及行政费用,折旧与摊销,资本性支出
2016,3886219.00,247590.94,586824.18,93419.47,101917.81
2017,5821786.13,481460.19,770661.13,112583.45,112501.72
2018,7363887.24,534749.63,789801.77,117542.55,160675.02


In [13]:
#计算资本性支出平均增长率
capitalzhichu=total2['资本性支出']/total2['营业收入']
meanzhichu=capitalzhichu.mean()
meanzhichu

0.02245634157731856

In [14]:
#营运资本估算，筛选相关科目
capital1=data1[(data1['Accounts']=='流动资产合计')|
              (data1['Accounts']=='应收账款')|
              (data1['Accounts']=='存货')|
              (data1['Accounts']=='流动负债合计')|
              (data1['Accounts']=='应付账款')]
capital1

,Accounts,2016,2017,2018
5,应收账款,0.00,0.00,0.00
11,存货,2062225.18,2205748.14,2350695.08
17,流动资产合计,9018054.88,11224918.60,13786183.53
44,应付账款,104060.82,99205.59,117829.64
57,流动负债合计,3702042.54,3857491.94,4243818.68


In [15]:
#计算相关指标
capital2=np.array(capital1.iloc[:,[1,2,3]])
total['应收账款']=capital2[0]
total['存货']=capital2[1]
total['流动资产合计']=capital2[2]
total['应付账款']=capital2[3]
total['流动负债合计']=capital2[4]
total['应收账款周转天数']=total['应收账款']/total['营业收入']*365
total['存货周转天数']=total['存货']/total['不包括折旧与摊销的营业成本']*365
total['其他流动资产占销售收入']=(total['流动资产合计']-total['应收账款']-total['存货'])/total['营业收入']
total['应付账款周转天数']=total['应付账款']/total['不包括折旧与摊销的营业成本']*365
total['其他流动负债占销货成本']=(total['流动负债合计']-total['应付账款'])/total['不包括折旧与摊销的营业成本']
total

,营业收入,营业成本,销售费用,管理费用,折旧与摊销,不包括折旧与摊销的营业成本,销售管理以及行政费用,息前税前折旧摊销前利润（EBITDA）,EBIT,息前税后利润,应收账款,存货,流动资产合计,应付账款,流动负债合计,应收账款周转天数,存货周转天数,其他流动资产占销售收入,应付账款周转天数,其他流动负债占销货成本
2016,3886219.00,341010.41,168105.20,418718.98,93419.47,247590.94,586824.18,3051803.88,2958384.41,2.218788e+06,0.0,2062225.18,9018054.88,104060.82,3702042.54,0.0,3040.144323,1.789871,153.407064,14.531960
2017,5821786.13,594043.64,298606.85,472054.28,112583.45,481460.19,770661.13,4569664.81,4457081.36,3.342811e+06,0.0,2205748.14,11224918.60,99205.59,3857491.94,0.0,1672.200709,1.549210,75.208794,7.806017
2018,7363887.24,652292.18,257207.69,532594.08,117542.55,534749.63,789801.77,6039335.84,5921793.29,4.441345e+06,0.0,2350695.08,13786183.53,117829.64,4243818.68,0.0,1604.496116,1.552915,80.426084,7.715740


In [16]:
#计算一些指标的均值
meanyingshou=total['应收账款周转天数'].mean()
meancun=total['存货周转天数'].mean()
meanqitaliudong=total['其他流动资产占销售收入'].mean()
meanyingfu=total['应付账款周转天数'].mean()
meanqtfz=total['其他流动负债占销货成本'].mean()

In [17]:
#将一些科目合并进total2中
total2['应收账款']=capital2[0]
total2['存货']=capital2[1]
total2['其它流动资产合计']=total['流动资产合计']-total['应收账款']-total['存货']
total2['应付账款']=capital2[3]
total2['其他流动负债合计']=total['流动负债合计']-total['应付账款']
total2

,营业收入,不包括折旧与摊销的营业成本,销售管理以及行政费用,折旧与摊销,资本性支出,应收账款,存货,其它流动资产合计,应付账款,其他流动负债合计
2016,3886219.00,247590.94,586824.18,93419.47,101917.81,0.0,2062225.18,6955829.70,104060.82,3597981.72
2017,5821786.13,481460.19,770661.13,112583.45,112501.72,0.0,2205748.14,9019170.46,99205.59,3758286.35
2018,7363887.24,534749.63,789801.77,117542.55,160675.02,0.0,2350695.08,11435488.45,117829.64,4125989.04


In [18]:
for i in range(1,11):
    newprofit=total2.loc[year+i-1,'营业收入']*(1+meanprofit)#本期营业收入
    total2.loc[year+i]=[newprofit,
                        newprofit*meancost,#本期营业收入乘以营业成本占比
                        newprofit*meanfee,#本期营业收入乘以管理费用占比
                        newprofit*meanzhe,#本期营业收入乘以折旧与摊销占比
                        newprofit*meanzhichu,#本期资本性支出等于本期营业收入乘以资本性支出比重
                        newprofit/365*meanyingshou,#本期应收账款
                        newprofit*meancost/365*meancun,#本期存货
                        newprofit*meanqitaliudong,#本期其他流动资产合计
                        newprofit*meancost/365*meanyingfu,#营业收入乘以营业成本占比。再除365天，乘以应付账款周转天数
                        newprofit*meancost*meanqtfz]#其他流动负债，他等于本期营业成本乘以占比
total2

,营业收入,不包括折旧与摊销的营业成本,销售管理以及行政费用,折旧与摊销,资本性支出,应收账款,存货,其它流动资产合计,应付账款,其他流动负债合计
2016,3.886219e+06,2.475909e+05,5.868242e+05,9.341947e+04,1.019178e+05,0.0,2.062225e+06,6.955830e+06,1.040608e+05,3.597982e+06
2017,5.821786e+06,4.814602e+05,7.706611e+05,1.125835e+05,1.125017e+05,0.0,2.205748e+06,9.019170e+06,9.920559e+04,3.758286e+06
2018,7.363887e+06,5.347496e+05,7.898018e+05,1.175426e+05,1.606750e+05,0.0,2.350695e+06,1.143549e+07,1.178296e+05,4.125989e+06
2019,1.017300e+07,7.427227e+05,1.324627e+06,2.012185e+05,2.284484e+05,0.0,4.284622e+06,1.658876e+07,2.096187e+05,7.440526e+06
2020,1.405372e+07,1.026051e+06,1.829935e+06,2.779777e+05,3.155951e+05,0.0,5.919085e+06,2.291691e+07,2.895823e+05,1.027888e+07
2021,1.941482e+07,1.417460e+06,2.528004e+06,3.840184e+05,4.359857e+05,0.0,8.177049e+06,3.165906e+07,4.000498e+05,1.419998e+07
2022,2.682102e+07,1.958181e+06,3.492366e+06,5.305107e+05,6.023020e+05,0.0,1.129636e+07,4.373611e+07,5.526576e+05,1.961687e+07
2023,3.705249e+07,2.705172e+06,4.824605e+06,7.328856e+05,8.320633e+05,0.0,1.560561e+07,6.042020e+07,7.634809e+05,2.710016e+07
2024,5.118697e+07,3.737119e+06,6.665056e+06,1.012461e+06,1.149472e+06,0.0,2.155871e+07,8.346881e+07,1.054727e+06,3.743811e+07
2025,7.071336e+07,5.162725e+06,9.207588e+06,1.398686e+06,1.587963e+06,0.0,2.978275e+07,1.153098e+08,1.457076e+06,5.171969e+07


In [19]:
total2['EBIT']=total2['营业收入']-total2['不包括折旧与摊销的营业成本']-total2['销售管理以及行政费用']-total2['折旧与摊销']
total2['息前税后利润']=total2['EBIT']*(1-Tax)
total2

,营业收入,不包括折旧与摊销的营业成本,销售管理以及行政费用,折旧与摊销,资本性支出,应收账款,存货,其它流动资产合计,应付账款,其他流动负债合计,EBIT,息前税后利润
2016,3.886219e+06,2.475909e+05,5.868242e+05,9.341947e+04,1.019178e+05,0.0,2.062225e+06,6.955830e+06,1.040608e+05,3.597982e+06,2.958384e+06,2.218788e+06
2017,5.821786e+06,4.814602e+05,7.706611e+05,1.125835e+05,1.125017e+05,0.0,2.205748e+06,9.019170e+06,9.920559e+04,3.758286e+06,4.457081e+06,3.342811e+06
2018,7.363887e+06,5.347496e+05,7.898018e+05,1.175426e+05,1.606750e+05,0.0,2.350695e+06,1.143549e+07,1.178296e+05,4.125989e+06,5.921793e+06,4.441345e+06
2019,1.017300e+07,7.427227e+05,1.324627e+06,2.012185e+05,2.284484e+05,0.0,4.284622e+06,1.658876e+07,2.096187e+05,7.440526e+06,7.904435e+06,5.928326e+06
2020,1.405372e+07,1.026051e+06,1.829935e+06,2.779777e+05,3.155951e+05,0.0,5.919085e+06,2.291691e+07,2.895823e+05,1.027888e+07,1.091975e+07,8.189816e+06
2021,1.941482e+07,1.417460e+06,2.528004e+06,3.840184e+05,4.359857e+05,0.0,8.177049e+06,3.165906e+07,4.000498e+05,1.419998e+07,1.508533e+07,1.131400e+07
2022,2.682102e+07,1.958181e+06,3.492366e+06,5.305107e+05,6.023020e+05,0.0,1.129636e+07,4.373611e+07,5.526576e+05,1.961687e+07,2.083996e+07,1.562997e+07
2023,3.705249e+07,2.705172e+06,4.824605e+06,7.328856e+05,8.320633e+05,0.0,1.560561e+07,6.042020e+07,7.634809e+05,2.710016e+07,2.878982e+07,2.159237e+07
2024,5.118697e+07,3.737119e+06,6.665056e+06,1.012461e+06,1.149472e+06,0.0,2.155871e+07,8.346881e+07,1.054727e+06,3.743811e+07,3.977233e+07,2.982925e+07
2025,7.071336e+07,5.162725e+06,9.207588e+06,1.398686e+06,1.587963e+06,0.0,2.978275e+07,1.153098e+08,1.457076e+06,5.171969e+07,5.494436e+07,4.120827e+07


In [20]:
total2['营运资本']=total2['应收账款']+total2['存货']+total2['其它流动资产合计']-total2['其他流动负债合计']-total2['应付账款']
total2['营运资本增加额']=total2['营运资本']-total2['营运资本'].shift(1)#下一行减去上一行的值
total2

,营业收入,不包括折旧与摊销的营业成本,销售管理以及行政费用,折旧与摊销,资本性支出,应收账款,存货,其它流动资产合计,应付账款,其他流动负债合计,EBIT,息前税后利润,营运资本,营运资本增加额
2016,3.886219e+06,2.475909e+05,5.868242e+05,9.341947e+04,1.019178e+05,0.0,2.062225e+06,6.955830e+06,1.040608e+05,3.597982e+06,2.958384e+06,2.218788e+06,5.316012e+06,NaN
2017,5.821786e+06,4.814602e+05,7.706611e+05,1.125835e+05,1.125017e+05,0.0,2.205748e+06,9.019170e+06,9.920559e+04,3.758286e+06,4.457081e+06,3.342811e+06,7.367427e+06,2.051414e+06
2018,7.363887e+06,5.347496e+05,7.898018e+05,1.175426e+05,1.606750e+05,0.0,2.350695e+06,1.143549e+07,1.178296e+05,4.125989e+06,5.921793e+06,4.441345e+06,9.542365e+06,2.174938e+06
2019,1.017300e+07,7.427227e+05,1.324627e+06,2.012185e+05,2.284484e+05,0.0,4.284622e+06,1.658876e+07,2.096187e+05,7.440526e+06,7.904435e+06,5.928326e+06,1.322324e+07,3.680875e+06
2020,1.405372e+07,1.026051e+06,1.829935e+06,2.779777e+05,3.155951e+05,0.0,5.919085e+06,2.291691e+07,2.895823e+05,1.027888e+07,1.091975e+07,8.189816e+06,1.826753e+07,5.044294e+06
2021,1.941482e+07,1.417460e+06,2.528004e+06,3.840184e+05,4.359857e+05,0.0,8.177049e+06,3.165906e+07,4.000498e+05,1.419998e+07,1.508533e+07,1.131400e+07,2.523608e+07,6.968550e+06
2022,2.682102e+07,1.958181e+06,3.492366e+06,5.305107e+05,6.023020e+05,0.0,1.129636e+07,4.373611e+07,5.526576e+05,1.961687e+07,2.083996e+07,1.562997e+07,3.486294e+07,9.626856e+06
2023,3.705249e+07,2.705172e+06,4.824605e+06,7.328856e+05,8.320633e+05,0.0,1.560561e+07,6.042020e+07,7.634809e+05,2.710016e+07,2.878982e+07,2.159237e+07,4.816217e+07,1.329923e+07
2024,5.118697e+07,3.737119e+06,6.665056e+06,1.012461e+06,1.149472e+06,0.0,2.155871e+07,8.346881e+07,1.054727e+06,3.743811e+07,3.977233e+07,2.982925e+07,6.653468e+07,1.837251e+07
2025,7.071336e+07,5.162725e+06,9.207588e+06,1.398686e+06,1.587963e+06,0.0,2.978275e+07,1.153098e+08,1.457076e+06,5.171969e+07,5.494436e+07,4.120827e+07,9.191579e+07,2.538111e+07


In [21]:
total3=total2.loc[:,['息前税后利润','折旧与摊销','资本性支出','营运资本增加额']]
total3

,息前税后利润,折旧与摊销,资本性支出,营运资本增加额
2016,2.218788e+06,9.341947e+04,1.019178e+05,NaN
2017,3.342811e+06,1.125835e+05,1.125017e+05,2.051414e+06
2018,4.441345e+06,1.175426e+05,1.606750e+05,2.174938e+06
2019,5.928326e+06,2.012185e+05,2.284484e+05,3.680875e+06
2020,8.189816e+06,2.779777e+05,3.155951e+05,5.044294e+06
2021,1.131400e+07,3.840184e+05,4.359857e+05,6.968550e+06
2022,1.562997e+07,5.305107e+05,6.023020e+05,9.626856e+06
2023,2.159237e+07,7.328856e+05,8.320633e+05,1.329923e+07
2024,2.982925e+07,1.012461e+06,1.149472e+06,1.837251e+07
2025,4.120827e+07,1.398686e+06,1.587963e+06,2.538111e+07


In [22]:
total3['实体现金流量']=total3['息前税后利润']+total3['折旧与摊销']-total3['资本性支出']-total3['营运资本增加额']
total3

,息前税后利润,折旧与摊销,资本性支出,营运资本增加额,实体现金流量
2016,2.218788e+06,9.341947e+04,1.019178e+05,NaN,NaN
2017,3.342811e+06,1.125835e+05,1.125017e+05,2.051414e+06,1.291478e+06
2018,4.441345e+06,1.175426e+05,1.606750e+05,2.174938e+06,2.223274e+06
2019,5.928326e+06,2.012185e+05,2.284484e+05,3.680875e+06,2.220221e+06
2020,8.189816e+06,2.779777e+05,3.155951e+05,5.044294e+06,3.107905e+06
2021,1.131400e+07,3.840184e+05,4.359857e+05,6.968550e+06,4.293483e+06
2022,1.562997e+07,5.305107e+05,6.023020e+05,9.626856e+06,5.931326e+06
2023,2.159237e+07,7.328856e+05,8.320633e+05,1.329923e+07,8.193959e+06
2024,2.982925e+07,1.012461e+06,1.149472e+06,1.837251e+07,1.131972e+07
2025,4.120827e+07,1.398686e+06,1.587963e+06,2.538111e+07,1.563788e+07


无风险利率，这里使用同到期期限的国债到期收益率。这里数据我们主要从和讯财经网上获取。

使用以下程序来获取无风险利率，由于每一年1月1日均为节假期，没有数据，为了避免出现空缺值，我们使用1月4日的国债到期收益率曲线，获取到期期限为一年到期收益率

In [23]:
#获取无风险利率，由于每一年1月1日均为节假期，没有数据，为了避免出现空缺值，我们使用1月4日的国债到期收益率曲线，
#并主要获取到期期限为一年的数据
def RF(year):
    date=str(year)+'-01-04'
    Rfurl='http://bond.money.hexun.com/data/js/data/trans.aspx?date='+str(year)+'-01-04&count=19&titType=null&page=1'
    from_data={'date':date,
           'count':'19',
           'titType':'null','page':'1'}
    header1={'Host':'bond.money.hexun.com',
         'Referer':'http://bond.money.hexun.com/data/yield.aspx',
         'User-Agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) \
         AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36 SE 2.X MetaSr 1.0'}
    response1=requests.get(Rfurl,headers=header1,data=from_data)
    re2=response1.text
    re3=re.findall("standardDeadline:'1y',yield:'(.*?)'}",re2)
    Rf=float(re3[0])
    return Rf
RF(year)

3.5968